In [1]:
import pandas as pd 
import xarray as xr

## Energy data

In [2]:
energy_data_20200920_20240118 = pd.read_csv("HEFTcom24/data/energy_data/Energy_Data_20200920_20240118.csv")
energy_data_20240119_20240519 = pd.read_csv("HEFTcom24/data/energy_data/energy_data_20240119_20240519.csv")

In [3]:
energy_data_old = energy_data_20200920_20240118.copy()
energy_data_new = energy_data_20240119_20240519.copy()

energy_data_old["dtm"] = pd.to_datetime(energy_data_old["dtm"])
energy_data_old["Wind_MWh_credit"] = 0.5 * energy_data_old["Wind_MW"] - energy_data_old["boa_MWh"]
energy_data_old["Solar_MWh_credit"] = 0.5 * energy_data_old["Solar_MW"]

energy_data_new["dtm"] = pd.to_datetime(energy_data_new["dtm"])
energy_data_new["Wind_MWh_credit"] = 0.5 * energy_data_new["Wind_MW"] - energy_data_new["boa_MWh"]
energy_data_new["Solar_MWh_credit"] = 0.5 * energy_data_new["Solar_MW"]

energy_data_merged = pd.concat([energy_data_old, energy_data_new], ignore_index=True)

In [4]:
energy_data = energy_data_merged.copy()

In [5]:
energy_data.head()

,dtm,MIP,Solar_MW,Solar_capacity_mwp,Solar_installedcapacity_mwp,Wind_MW,SS_Price,boa_MWh,DA_Price,Wind_MWh_credit,Solar_MWh_credit
0,2020-09-20 00:00:00+00:00,20.06,0.0,2130.537493,2228.208777,996.284,2.50,0.0,32.17,498.142,0.0
1,2020-09-20 00:30:00+00:00,19.77,0.0,2130.537493,2228.208777,957.576,15.00,0.0,32.17,478.788,0.0
2,2020-09-20 01:00:00+00:00,28.68,0.0,2130.537493,2228.208777,941.044,47.95,0.0,32.00,470.522,0.0
3,2020-09-20 01:30:00+00:00,28.97,0.0,2130.537493,2228.208777,964.366,29.13,0.0,32.00,482.183,0.0
4,2020-09-20 02:00:00+00:00,28.19,0.0,2130.537493,2228.208777,918.432,28.95,0.0,31.99,459.216,0.0


In [6]:
print(energy_data["MIP"].isna().sum())
print(energy_data["Solar_MW"].isna().sum())
print(energy_data["Wind_MW"].isna().sum())
print(energy_data["boa_MWh"].isna().sum())
print(energy_data["Wind_MWh_credit"].isna().sum())
print(energy_data["Solar_MWh_credit"].isna().sum())
print(energy_data["Solar_capacity_mwp"].isna().sum())
print(energy_data["Solar_installedcapacity_mwp"].isna().sum())
print(energy_data["SS_Price"].isna().sum())
print(energy_data["DA_Price"].isna().sum())

425
75
103
103
103
75
0
0
0
0


In [7]:
energy_data = energy_data.dropna(subset= ["MIP", "Solar_MW", "Wind_MW", "boa_MWh", "Wind_MWh_credit", "Solar_MWh_credit", "Solar_capacity_mwp", "Solar_installedcapacity_mwp", "SS_Price", "DA_Price"])

## Weather forecasts

In [8]:
import os
import pandas as pd
import xarray as xr

def process_and_concat_files(directory):
    dfs = []

    for file in os.listdir(directory):
        if file.endswith(".nc"):
            file_path = os.path.join(directory, file)
            ds = xr.open_dataset(file_path)
            df = ds.to_dataframe().reset_index()
            
            if "ref_datetime" in df.columns and "valid_datetime" in df.columns:
                df.rename(columns={"ref_datetime": "reference_time", "valid_datetime": "valid_time"}, inplace=True)
            elif "reference_time" not in df.columns or "valid_time" not in df.columns:
                raise ValueError("Neither 'ref_datetime' and 'valid_datetime' nor 'reference_time' and 'valid_time' found in the dataset.")
            
            df["reference_time"] = df["reference_time"].dt.tz_localize("UTC")
            df["valid_time"] = df["reference_time"] + pd.to_timedelta(df["valid_time"], unit="hours")
            df["reference_time"] = df["reference_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
            df["valid_time"] = df["valid_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
            dfs.append(df)
    
    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df["reference_time"] = pd.to_datetime(concatenated_df["reference_time"])
    concatenated_df["valid_time"] = pd.to_datetime(concatenated_df["valid_time"])
    concatenated_df["reference_time"] = concatenated_df["reference_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    concatenated_df["valid_time"] = concatenated_df["valid_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    
    return concatenated_df

In [49]:
base_dir = "HEFTcom24/data"

# dwd_demand_df = process_and_concat_files(os.path.join(base_dir, "dwd_demand"))
# dwd_pes10_df = process_and_concat_files(os.path.join(base_dir, "dwd_pes10"))
dwd_hornsea_1_df = process_and_concat_files(os.path.join(base_dir, "dwd_hornsea_1"))

# ncep_demand_df = process_and_concat_files(os.path.join(base_dir, "ncep_demand"))
# ncep_pes10_df = process_and_concat_files(os.path.join(base_dir, "ncep_pes10"))
ncep_hornsea_1_df = process_and_concat_files(os.path.join(base_dir, "ncep_hornsea_1"))

## HORNSEA_1

In [50]:
dwd_hornsea_1_df.head()

,reference_time,valid_time,latitude,longitude,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100
0,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.702,59.105434,4.916626,54.525970,54.250305,6.547671,6.854635
1,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.767,58.628872,4.860962,54.288940,54.106964,6.524992,6.817597
2,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.832,58.365200,4.823853,53.520721,53.448120,6.493145,6.795032
3,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.897,58.441372,4.814087,52.447235,52.500397,6.495604,6.822487
4,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.962,58.501919,4.804321,51.579865,51.684021,6.521748,6.856290


In [51]:
print(dwd_hornsea_1_df["RelativeHumidity"].isna().sum())
print(dwd_hornsea_1_df["Temperature"].isna().sum())
print(dwd_hornsea_1_df["WindDirection"].isna().sum())
print(dwd_hornsea_1_df["WindDirection:100"].isna().sum())
print(dwd_hornsea_1_df["WindSpeed"].isna().sum())
print(dwd_hornsea_1_df["WindSpeed:100"].isna().sum())

print(ncep_hornsea_1_df["RelativeHumidity"].isna().sum())
print(ncep_hornsea_1_df["Temperature"].isna().sum())
print(ncep_hornsea_1_df["WindDirection"].isna().sum())
print(ncep_hornsea_1_df["WindDirection:100"].isna().sum())
print(ncep_hornsea_1_df["WindSpeed"].isna().sum())
print(ncep_hornsea_1_df["WindSpeed:100"].isna().sum())

13356
13392
13356
12204
13356
12204
3456
3627
5733
5733
5733
5733


In [52]:
dwd_hornsea_1_df.dropna(subset=["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"], inplace=True)
ncep_hornsea_1_df.dropna(subset=["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"], inplace=True)

In [53]:
import numpy as np

dwd_hornsea_1_df["WindSpeed^3"] = dwd_hornsea_1_df["WindSpeed"] ** 3
dwd_hornsea_1_df["WindSpeed:100^3"] = dwd_hornsea_1_df["WindSpeed:100"] ** 3

ncep_hornsea_1_df["WindSpeed^3"] = ncep_hornsea_1_df["WindSpeed"] ** 3
ncep_hornsea_1_df["WindSpeed:100^3"] = ncep_hornsea_1_df["WindSpeed:100"] ** 3

dwd_h1df_mean = dwd_hornsea_1_df.groupby(["reference_time", "valid_time"])[["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed^3", "WindSpeed:100^3"]].mean().reset_index()
ncep_h1df_mean = ncep_hornsea_1_df.groupby(["reference_time", "valid_time"])[["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed^3", "WindSpeed:100^3"]].mean().reset_index()

dwd_h1df_mean["WindSpeed"] = np.cbrt(dwd_h1df_mean["WindSpeed^3"])
dwd_h1df_mean["WindSpeed:100"] = np.cbrt(dwd_h1df_mean["WindSpeed:100^3"])

ncep_h1df_mean["WindSpeed"] = np.cbrt(ncep_h1df_mean["WindSpeed^3"])
ncep_h1df_mean["WindSpeed:100"] = np.cbrt(ncep_h1df_mean["WindSpeed:100^3"])

dwd_h1df_mean = dwd_h1df_mean.drop(columns=["WindSpeed^3", "WindSpeed:100^3"])
ncep_h1df_mean = ncep_h1df_mean.drop(columns=["WindSpeed^3", "WindSpeed:100^3"])

In [54]:
merged_h1_df = dwd_h1df_mean.merge(ncep_h1df_mean, on=["reference_time", "valid_time"], suffixes=("_dwd", "_ncep"), how= "outer")
merged_h1_df["reference_time"] = pd.to_datetime(merged_h1_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
merged_h1_df["valid_time"] = pd.to_datetime(merged_h1_df["valid_time"], format="%Y-%m-%d %H:%M:%S")

In [56]:
merged_h1_df = merged_h1_df.set_index("valid_time").groupby("reference_time").resample("30min").interpolate("linear")

In [57]:
merged_h1_df = merged_h1_df.drop(columns=["reference_time"], axis= 1).reset_index()

In [58]:
merged_h1_df["valid_time"] = pd.to_datetime(merged_h1_df["valid_time"], format="%Y-%m-%d %H:%M:%S", utc= True)

In [59]:
merged_h1_energy_data = merged_h1_df.merge(energy_data, how= "inner", left_on= "valid_time", right_on= "dtm")

In [60]:
merged_h1_energy_data = merged_h1_energy_data.sort_values(by= "valid_time")
merged_h1_energy_data = merged_h1_energy_data.groupby("valid_time").tail(1).reset_index()
merged_h1_energy_data["valid_time"] = pd.to_datetime(merged_h1_energy_data["valid_time"], format="%Y-%m-%d %H:%M:%S", utc= True)

In [63]:
merged_h1_energy_data = merged_h1_energy_data.drop(columns=["index", "Solar_MW", "Solar_MWh_credit", "Solar_capacity_mwp", "Solar_installedcapacity_mwp"], axis= 1)

In [64]:
merged_h1_energy_data.to_csv("HEFTcom24/data/wind1.csv", index= False)

## PES10

In [55]:
test1 = dwd_pes10_df.drop_duplicates(subset=["longitude", "latitude"])[["longitude", "latitude"]]
test1.to_clipboard(index=False)

In [29]:
dwd_pes10_df.head()

,reference_time,valid_time,point,CloudCover,SolarDownwardRadiation,Temperature,latitude,longitude
0,2024-01-08 00:00:00,2024-01-08 00:00:00,0,0.409668,0.0,0.316040,52.487256,0.401245
1,2024-01-08 00:00:00,2024-01-08 00:00:00,1,0.508945,0.0,1.136353,52.877668,0.790653
2,2024-01-08 00:00:00,2024-01-08 00:00:00,2,0.546328,0.0,0.637329,52.135428,-0.264034
3,2024-01-08 00:00:00,2024-01-08 00:00:00,3,0.302520,0.0,0.057251,52.488050,-0.126705
4,2024-01-08 00:00:00,2024-01-08 00:00:00,4,0.621309,0.0,1.159790,51.956370,0.658817


In [30]:
print(dwd_pes10_df["SolarDownwardRadiation"].isna().sum())
print(dwd_pes10_df["Temperature"].isna().sum())
print(dwd_pes10_df["CloudCover"].isna().sum())

print(ncep_pes10_df["SolarDownwardRadiation"].isna().sum())
print(ncep_pes10_df["Temperature"].isna().sum())
print(ncep_pes10_df["CloudCover"].isna().sum())

126920
7440
7440
104180
10173
15313


In [4]:
dwd_pes10_df = dwd_pes10_df.dropna(subset= ["Temperature", "SolarDownwardRadiation", "CloudCover"])
ncep_pes10_df = ncep_pes10_df.dropna(subset= ["Temperature", "SolarDownwardRadiation", "CloudCover"])

In [5]:
pivot_dwd_pes10_df = dwd_pes10_df.pivot_table(
    index=["valid_time", "reference_time"],
    columns="point",
    values=["Temperature", "CloudCover", "SolarDownwardRadiation"]
)
pivot_dwd_pes10_df.columns = ['_Point'.join(map(str, col)).strip() for col in pivot_dwd_pes10_df.columns.values]
pivot_dwd_pes10_df.reset_index(inplace=True)

pivot_ncep_pes10_df = ncep_pes10_df.pivot_table(
    index=["valid_time", "reference_time"],
    columns="point",
    values=["Temperature", "CloudCover", "SolarDownwardRadiation"]
)
pivot_ncep_pes10_df.columns = ['_Point'.join(map(str, col)).strip() for col in pivot_ncep_pes10_df.columns.values]
pivot_ncep_pes10_df.reset_index(inplace=True)


In [6]:
# pivot_dwd_pes10_df["Mean_CloudCover"] = pivot_dwd_pes10_df.filter(like="CloudCover").mean(axis=1)
# pivot_dwd_pes10_df["Mean_Temperature"] = pivot_dwd_pes10_df.filter(like="Temperature").mean(axis=1)
# pivot_dwd_pes10_df["Mean_SolarDownwardRadiation"] = pivot_dwd_pes10_df.filter(like="SolarDownwardRadiation").mean(axis=1)

# pivot_dwd_pes10_df["Std_CloudCover"] = pivot_dwd_pes10_df.filter(like="CloudCover").std(axis=1)
# pivot_dwd_pes10_df["Std_Temperature"] = pivot_dwd_pes10_df.filter(like="Temperature").std(axis=1)
# pivot_dwd_pes10_df["Std_SolarDownwardRadiation"] = pivot_dwd_pes10_df.filter(like="SolarDownwardRadiation").std(axis=1)

pivot_dwd_pes10_df["reference_time"] = pd.to_datetime(pivot_dwd_pes10_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
pivot_dwd_pes10_df["valid_time"] = pd.to_datetime(pivot_dwd_pes10_df["valid_time"], format="%Y-%m-%d %H:%M:%S")


# pivot_ncep_pes10_df["Mean_CloudCover"] = pivot_ncep_pes10_df.filter(like="CloudCover").mean(axis=1)
# pivot_ncep_pes10_df["Mean_Temperature"] = pivot_ncep_pes10_df.filter(like="Temperature").mean(axis=1)
# pivot_ncep_pes10_df["Mean_SolarDownwardRadiation"] = pivot_ncep_pes10_df.filter(like="SolarDownwardRadiation").mean(axis=1)

# pivot_ncep_pes10_df["Std_CloudCover"] = pivot_ncep_pes10_df.filter(like="CloudCover").std(axis=1)
# pivot_ncep_pes10_df["Std_Temperature"] = pivot_ncep_pes10_df.filter(like="Temperature").std(axis=1)
# pivot_ncep_pes10_df["Std_SolarDownwardRadiation"] = pivot_ncep_pes10_df.filter(like="SolarDownwardRadiation").std(axis=1)

pivot_ncep_pes10_df["reference_time"] = pd.to_datetime(pivot_ncep_pes10_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
pivot_ncep_pes10_df["valid_time"] = pd.to_datetime(pivot_ncep_pes10_df["valid_time"], format="%Y-%m-%d %H:%M:%S")

In [7]:
merged_pes10_df = pivot_dwd_pes10_df.merge(pivot_ncep_pes10_df, on=["reference_time", "valid_time"], suffixes=("_dwd", "_ncep"), how= "outer")

In [9]:
merged_pes10_df.columns.values

array(['valid_time', 'reference_time', 'CloudCover_Point0_dwd',
       'CloudCover_Point1_dwd', 'CloudCover_Point2_dwd',
       'CloudCover_Point3_dwd', 'CloudCover_Point4_dwd',
       'CloudCover_Point5_dwd', 'CloudCover_Point6_dwd',
       'CloudCover_Point7_dwd', 'CloudCover_Point8_dwd',
       'CloudCover_Point9_dwd', 'CloudCover_Point10_dwd',
       'CloudCover_Point11_dwd', 'CloudCover_Point12_dwd',
       'CloudCover_Point13_dwd', 'CloudCover_Point14_dwd',
       'CloudCover_Point15_dwd', 'CloudCover_Point16_dwd',
       'CloudCover_Point17_dwd', 'CloudCover_Point18_dwd',
       'CloudCover_Point19_dwd', 'SolarDownwardRadiation_Point0_dwd',
       'SolarDownwardRadiation_Point1_dwd',
       'SolarDownwardRadiation_Point2_dwd',
       'SolarDownwardRadiation_Point3_dwd',
       'SolarDownwardRadiation_Point4_dwd',
       'SolarDownwardRadiation_Point5_dwd',
       'SolarDownwardRadiation_Point6_dwd',
       'SolarDownwardRadiation_Point7_dwd',
       'SolarDownwardRadiation_Point

## Demand

In [36]:
ncep_demand_df.head()

,reference_time,valid_time,point,RelativeHumidity,Temperature,TotalPrecipitation,WindDirection,WindSpeed,latitude,longitude
0,2020-09-20 00:00:00,2020-09-20 00:00:00,0,74.300003,15.350006,0.0,38.035126,4.788564,51.479,-0.451
1,2020-09-20 00:00:00,2020-09-20 00:00:00,1,83.800003,12.750000,0.0,35.796936,5.266498,51.453,-2.600
2,2020-09-20 00:00:00,2020-09-20 00:00:00,2,88.599998,11.250000,0.0,40.649506,4.298924,52.449,-1.926
3,2020-09-20 00:00:00,2020-09-20 00:00:00,3,84.599998,10.350006,0.0,78.319733,1.144141,53.175,-2.986
4,2020-09-20 00:00:00,2020-09-20 00:00:00,4,87.000000,9.050018,0.0,65.584869,2.350657,55.860,-4.264


In [58]:
import folium

map = ncep_demand_df.drop_duplicates(["longitude", "latitude"])[["longitude", "latitude"]]

m = folium.Map(location=[20, 0], zoom_start=2)

# Füge Marker für jeden Punkt hinzu
for idx, row in map.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']]
    ).add_to(m)

#m

In [37]:
print(ncep_demand_df["RelativeHumidity"].isna().sum())
print(ncep_demand_df["Temperature"].isna().sum())
print(ncep_demand_df["WindDirection"].isna().sum())
print(ncep_demand_df["WindSpeed"].isna().sum())
print(ncep_demand_df["WindDirection"].isna().sum())
print(ncep_demand_df["TotalPrecipitation"].isna().sum())


print(dwd_demand_df["Temperature"].isna().sum())
print(dwd_demand_df["RelativeHumidity"].isna().sum())
print(dwd_demand_df["WindDirection"].isna().sum())
print(dwd_demand_df["WindSpeed"].isna().sum())
print(dwd_demand_df["WindDirection"].isna().sum())
print(dwd_demand_df["TotalPrecipitation"].isna().sum())

4987
3657
5922
5922
5922
1956
2604
2597
2597
2597
2597
76615


In [38]:
ncep_demand_df = ncep_demand_df.dropna(subset= ["Temperature", "RelativeHumidity", "WindDirection", "WindSpeed", "TotalPrecipitation"])
dwd_demand_df = dwd_demand_df.dropna(subset= ["Temperature", "RelativeHumidity", "WindSpeed", "WindDirection", "TotalPrecipitation"])

In [39]:
pivot_ncep_demand_df = ncep_demand_df.pivot_table(
    index=["valid_time", "reference_time"],
    columns="point",
    values=["Temperature", "RelativeHumidity", "WindDirection", "WindSpeed", "TotalPrecipitation"]
)
pivot_ncep_demand_df.columns = ['_Point'.join(map(str, col)).strip() for col in pivot_ncep_demand_df.columns.values]
pivot_ncep_demand_df.reset_index(inplace=True)


pivot_dwd_demand_df = dwd_demand_df.pivot_table(
    index=["valid_time", "reference_time"],
    columns="point",
    values=["Temperature", "RelativeHumidity", "WindDirection", "WindSpeed", "TotalPrecipitation"]
)
pivot_dwd_demand_df.columns = ['_Point'.join(map(str, col)).strip() for col in pivot_dwd_demand_df.columns.values]
pivot_dwd_demand_df.reset_index(inplace=True)

In [40]:
pivot_dwd_demand_df["reference_time"] = pd.to_datetime(pivot_dwd_demand_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
pivot_dwd_demand_df["valid_time"] = pd.to_datetime(pivot_dwd_demand_df["valid_time"], format="%Y-%m-%d %H:%M:%S")

pivot_ncep_demand_df["reference_time"] = pd.to_datetime(pivot_ncep_demand_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
pivot_ncep_demand_df["valid_time"] = pd.to_datetime(pivot_ncep_demand_df["valid_time"], format="%Y-%m-%d %H:%M:%S")

In [41]:
merged_demand_df = pivot_dwd_demand_df.merge(pivot_ncep_demand_df, on=["valid_time", "reference_time"], suffixes=("_dwd", "_ncep"), how="outer")

## Merger

In [42]:
merged_weather_dwd_ncep_forecasts = merged_pes10_df.merge(merged_demand_df, on=["valid_time", "reference_time"], how="outer", suffixes=("_pes10", "_demand"))
merged_weather_dwd_ncep_forecasts = merged_weather_dwd_ncep_forecasts.merge(merged_h1_df, on=["valid_time", "reference_time"], how="outer")

In [43]:
#merged_weather_dwd_ncep_forecasts_res = merged_weather_dwd_ncep_forecasts.set_index("valid_time").groupby("reference_time").resample("30min").interpolate("linear")

In [44]:
#merged_weather_dwd_ncep_forecasts_res = merged_weather_dwd_ncep_forecasts_res.drop(columns= "reference_time",axis=1).reset_index()

In [45]:
#merged_weather_dwd_ncep_forecasts_res["valid_time"] = pd.to_datetime(merged_weather_dwd_ncep_forecasts_res["valid_time"], format="%Y-%m-%d %H:%M:%S", utc= True)

In [46]:
import pickle 

#merged_weather_dwd_ncep_forecasts_res.to_pickle("merged_weather_dwd_ncep_forecasts_res_30min.pkl")

In [47]:
#merged_dataset = merged_weather_dwd_ncep_forecasts_res.merge(energy_data, how= "inner", left_on= "valid_time", right_on="dtm")

In [48]:
# import numpy as np

# merged_dataset["valid_time"] = pd.to_datetime(merged_dataset["valid_time"], format= "%Y-%m-%d %H:%M:%S", utc=True)
# merged_dataset["reference_time"] = pd.to_datetime(merged_dataset["reference_time"], format= "%Y-%m-%d %H:%M:%S", utc=True)

# merged_dataset = merged_dataset[merged_dataset["valid_time"] - merged_dataset["reference_time"] < np.timedelta64(50, "h")]

In [49]:
#merged_dataset.to_pickle("merged_dataset_prep_utc.pkl")

In [50]:
#merged_dataset.to_csv("merged_dataset_prep_utc.csv")